In [1]:
# Load Packages
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import os
import geopandas as gpd
import regionmask
import oisstools as ot
import cartopy.crs as ccrs


# Set the workspace - local/ docker
workspace = "local"
box_root = ot.set_workspace(workspace)

# document what choices were made on last run:
print(f"Working via directory at: {box_root}")

Working via directory at: /Users/akemberling/Library/CloudStorage/Box-Box/


In [4]:
# Specify year range to update
yr_min = 1995
yr_max = 2024

# Set desired climatology period
reference_period = "1991-2020"
print(f"Calculating Anomalies for {reference_period} reference period.")

Calculating Anomalies for 1991-2020 reference period.


# Run Global Timeseries

In [5]:
ot.update_global_timeseries(
    yr_min, 
    yr_max, 
    box_root, 
    var_name = "sst", 
    reference_period = reference_period, 
    append_existing=True)

Updating Global Timeseries


---

# Manual Coding Section

the following code was used to develop the above function and can be useful for debugging:

In [3]:
# # Load all years of oisst and anomalies
# oisst = ot.load_box_oisst(box_root, 
#                           yr_min, 
#                           yr_max, 
#                           anomalies = False, 
#                           do_parallel = True)
# oisst = ot.add_mod(oisst, 'time')

# # Anomalies
# daily_anoms = ot.load_box_oisst(box_root, 
#                           yr_min, 
#                           yr_max, 
#                           anomalies = True, 
#                           do_parallel = True)

# # Climatology
# oisst_clim = ot.load_oisst_climatology(box_root = box_root, 
#                                        reference_period = reference_period)

In [7]:
# # Mean Temp
# mean_sst = oisst.mean(["lat", "lon"])
# weighted_sst = ot.area_weighted_means(oisst, var_name = "sst", sd = False)

# # Convert to dataframes
# sst_df     = mean_sst.to_dataframe().reset_index()
# sst_wt_df  = weighted_sst.to_dataframe().reset_index()

In [9]:
# # Merge standard and area weighted values
# sst_join  = sst_df.merge(sst_wt_df, how = "left", on = ["time", "MOD"])
# sst_join = sst_join.drop(columns=['modified_ordinal_day'])
# sst_join = sst_join[['time', 'MOD', 'sst', 'area_wtd_sst']]

,time,sst,modified_ordinal_day,MOD,area_wtd_sst
0,1981-09-01,13.792295,245.0,245,18.088272
1,1981-09-02,13.778812,246.0,246,18.078165
2,1981-09-03,13.771958,247.0,247,18.072233
3,1981-09-04,13.761700,248.0,248,18.066187
4,1981-09-05,13.745054,249.0,249,18.054379


In [20]:
# # Mean Climatology
# mean_clim = oisst_clim.mean(["lat", "lon"])
# weighted_clim = ot.area_weighted_means(oisst_clim, var_name = "sst", sd = False)

# clim_df    = mean_clim.to_dataframe().reset_index()
# clim_wt_df = weighted_clim.to_dataframe().reset_index()

# # climatology
# clim_join = clim_df.merge(clim_wt_df, on = "modified_ordinal_day")
# clim_join = clim_join.rename(columns = {"sst" : "sst_clim", f"area_wtd_sst" : "area_wtd_clim", "modified_ordinal_day" : "MOD"})

In [21]:
# # Join sst and climatology
# sst_and_clim = sst_join.merge(clim_join, how = "left", on = "MOD")

,MOD,sst_clim,area_wtd_clim
0,1,13.460504,17.978485
1,2,13.469034,17.986233
2,3,13.477575,17.994282
3,4,13.486596,18.003035
4,5,13.493990,18.010281


In [23]:
# # Mean Anomalies
# mean_anomalies = daily_anoms.mean(['lat', 'lon']) 
# weighted_means = ot.area_weighted_means(daily_anoms, var_name = "sst", sd = False)

# anom_df    = mean_anomalies.to_dataframe().reset_index().drop(columns = ["modified_ordinal_day"])
# anom_wt_df = weighted_means.to_dataframe().reset_index().drop(columns = ["modified_ordinal_day"])

# # Join anomalies
# anom_join = anom_df.merge(anom_wt_df, how = "left", on = ["time", "MOD"])
# anom_join = anom_join.rename(columns = {"sst" : "sst_anom", f"area_wtd_sst" : "area_wtd_anom"})

# Join SST + Clim + Anomalies

In [5]:
# update_ts = sst_and_clim.merge(anom_join, how = "left", on = ["time", "MOD"])

## Plots

In [ ]:
# mean_sst.sst.plot(label = "Global SST - Un-Weighted")
# weighted_sst.area_wtd_sst.plot(label = "Global SST - Area-Weighted")
# plt.legend()

# mean_clim.sst.plot(label = "Global SST - Un-Weighted")
# weighted_clim.area_wtd_sst.plot(label = "Global SST - Area-Weighted")
# plt.legend()

# # Plot Comparison
# mean_anomalies.sst.plot(label = "Global - Un-Weighted")
# weighted_means.area_wtd_sst.plot(label = "Global - Area-Weighted")
# plt.legend()

## Append to Full Timeseries

In [54]:
# # Open what we have already
# old_sst = pd.read_csv(f"{box_root}Res_Data/OISST/oisst_mainstays/global_timeseries/global_anoms_1982to2011.csv")
   
# # # Select/remove columns
# # old_sst = old_ts[["time", f"{var_name}", f"area_wtd_{var_name}"]]

# # Remove dates from old timeseries overlap from the update timeseries
# not_overlapped = ~old_sst.time.isin(update_ts.time)
# old_sst  = old_sst[not_overlapped]

# # Concatenate onto the original
# appended_ts = pd.concat([ old_sst, update_ts ])

# # Format time as datetime
# appended_ts["time"] = appended_ts["time"].astype("datetime64")

# # Sort
# appended_ts = appended_ts.sort_values(by = "time")

# # Drop duplicates
# appended_ts = appended_ts.drop_duplicates(subset=['time'])

# # Check
# appended_ts.head()

,time,MOD,sst,area_wtd_sst,sst_clim,area_wtd_clim,sst_anom,area_wtd_anom
0,1981-09-01,245,13.792295,18.088272,13.861716,18.187570,-0.069423,-0.099309
40,1981-09-02,246,13.778812,18.078165,13.860900,18.186693,-0.082091,-0.108535
80,1981-09-03,247,13.771958,18.072233,13.857845,18.184526,-0.085882,-0.112306
120,1981-09-04,248,13.761700,18.066187,13.853866,18.181871,-0.092171,-0.115656
160,1981-09-05,249,13.745054,18.054379,13.847774,18.177715,-0.102719,-0.123352


# Saving




In [41]:
# # SAVING
# appended_ts.to_csv(f"{box_root}Res_Data/OISST/oisst_mainstays/global_timeseries/global_anoms_1982to2011.csv", index = False)